In [ ]:
#default_exp unstructured2markup

In [ ]:
#export

def writeHeader(title, out, identifier=None):
    out.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")
    out.write("<!--" + title + "-->\n")
    out.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
    out.write("<volume>\n")
    out.write("<volumeTitle>" + title + "</volumeTitle>\n")
    if id != None:
        out.write("<volumeIdentifier>" + identifier + "</volumeIdentifier>")
    else:
        out.write("<volumeIdentifier />\n")
    out.write("<volumeGuid />\n")
    
    return

In [ ]:
#export

def writeFooter(out):
    out.write("</text>\n")
    out.write("</item>\n")
    out.write("</volume>\n")
    out.write("</ssda>\n")

    return

In [ ]:
#no_test

txt_path = input("Enter absolute path to txt version of trancription: ")
typ = input("Enter record type: ")
country = input("Enter country: ")
state = input("Enter first-level administrative division: ")
city = input("Enter city: ")
institution = input("Enter institution: ")
output_dir = input("Enter output directory: ")        
volume_identifier = input("Enter the volume's unique identifier: ")        
volume_title = input("Enter the volume's title: ")
img_type = input("Enter image file extensions: ")
curr_image_id = int(input("Enter first transcribed image file name: "))

first_folio = True
first_entry = True

with open(txt_path, encoding="utf-8") as inp:    
    output = open(output_dir,'w',encoding="utf-8")
    output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")    
    output.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
    output.write("<volume type=\"" + typ + "\" country=\"" + country + "\" state=\"" + state + "\" city=\"" + city + "\" institution=\"" + institution + "\" id=\"" + volume_identifier + "\" title=\"" + volume_title + "\">\n")

    in_entry = False
    for line in inp:
        if ("(" in line) and first_folio:
            first_folio = False
            output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\" number=\"1r\">\n")
            curr_image_id += 1
            curr_entry = 1
        elif len(line) < 3:
            continue
        elif '(' in line:            
            in_entry = False
            first_entry = True
            output.write("</entry>\n")
            output.write("</image>\n")
            output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\" number=\"" + line[:line.find('(')] + line[line.find('(') + 1:line.find(')')] + "\">\n")
            curr_image_id += 1
            curr_entry = 1
        elif line[0] == '.':
            if first_entry == True:
                first_entry = False
            else:
                output.write("</entry>\n")
            in_entry = True
            output.write("<entry id=\"" + str(curr_entry) + "\">\n")
            line = line.replace('<', '[')
            line = line.replace('>', ']')
            output.write(line[1:])
            curr_entry += 1       
        elif in_entry and not ("#####" in line):
            line = line.replace('<', '[')
            line = line.replace('>', ']')
            output.write(line)
        elif "#####" in line:
            output.write("</entry>\n")
            output.write("</image>\n")

    output.write("</volume>\n")
    output.write("</ssda>")

    inp.close()
    output.close() 

Enter absolute path to txt version of trancription: transcriptions\cart_baut_preprocess.txt
Enter record type: baptism
Enter country: Colombia
Enter first-level administrative division: Bolívar
Enter city: Cartagena de Indias
Enter institution: Iglesia de Santo Domingo
Enter output directory: transcriptions\cartagena_sd_baut_12.xml
Enter the volume's unique identifier: 0
Enter the volume's title: Libro de Bautismos No. 12
Enter image file extensions: jpg
Enter first transcribed image file name: 1008


In [ ]:
#no_test

#converts txt version of Genkins transcription into XML formatted according to SH specs, 
#originally built for DSI entity extraction project

txt_path = input("Enter absolute path to txt version of trancription: ")
volume_title = input("Enter volume title: ")
volume_id = input("Enter volume identifier: ")
output_path = input("Enter file name for output: ")

volume = open(txt_path, 'r', encoding="utf-8")
output = open(output_path + ".xml", 'w', encoding="utf-8")

writeHeader(volume_title, output, volume_id)

in_entry = False
first_folio = True
folio_no = 1001

for line in volume:
    if ("Fol" in line) and (first_folio == True):
        first_folio = False
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and (in_entry == False):
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
    elif line[:3] == "Fol" and in_entry:
        output.write("</entry>\n")
        output.write("</text>\n")
        output.write("</item>\n")
        output.write("<item>\n")
        output.write("<itemTitle>" + line[:line.find('\n')] + "</itemTitle>\n")
        output.write("<itemIdentifier>" + str(folio_no) + "</itemIdentifier>\n")
        folio_no += 1
        output.write("<itemGuid />\n")
        output.write("<text type=\"TranscribedText\">\n")
        output.write("<entry>\n")
    elif line[:3] == "[ma" and (in_entry == False):
        output.write("<entry>\n")
        output.write(line)
        in_entry = True
    elif in_entry and line[:3] == "[si":
        in_entry = False
        output.write(line)
        output.write("</entry>\n")
    elif line[:3] == "[ma":
        output.write("</entry>\n")
        output.write("<entry>\n")
        output.write(line)
    elif in_entry:
        line = line.replace('<', '')
        line = line.replace('>', '')
        output.write(line)

writeFooter(output)

volume.close()
output.close()

In [ ]:
#export

def universal_markup_converter(path_to_transcription, transcription_type):    
    if transcription_type == "spatial historian":
        inp = open(path_to_transcription,'r',encoding="utf-8")        
        transcription = ''
        for line in inp:
            transcription += line
        typ = input("Enter record type: ")
        country = input("Enter country: ")
        state = input("Enter first-level administrative division: ")
        city = input("Enter city: ")
        institution = input("Enter institution: ")
        output_dir = input("Enter output directory: ")
        vol_id_loc = transcription.find("<volumeIdentifier>")
        volume_identifier = transcription[vol_id_loc + len("<volumeIdentifier>"):transcription.find('<',vol_id_loc + 1)]
        vol_titl_loc = transcription.find("<volumeTitle>")
        volume_title = transcription[vol_titl_loc + len("<volumeTitle>"):transcription.find('<',vol_titl_loc + 1)]
        img_type = input("Enter image file extensions: ")
        curr_image_id = int(input("Enter first transcribed image file name: "))
        inp.close()
        inp = open(path_to_transcription,'r',encoding="utf-8")
        number_flag = input("Does the transcription contain meaningful folio numbers? (Y/N): ")
        
        output = open(output_dir,'w',encoding="utf-8")
        output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")    
        output.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
        output.write("<volume type=\"" + typ + "\" country=\"" + country + "\" state=\"" + state + "\" city=\"" + city + "\" institution=\"" + institution + "\" id=\"" + volume_identifier + "\" title=\"" + volume_title + "\">\n")
        
        in_entry = False
        for line in inp:
            if "<itemTitle>" in line:
                folio_start = line.find(' ')
                if number_flag == 'Y':
                    curr_image_number = line[line.find('>') + 1:line.find('<', line.find('<') + 1)]
                    if curr_image_number[:4] == "Fol_":
                        curr_image_number = curr_image_number[4:]
                    output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\" number=\"" + curr_image_number + "\">\n")
                else:
                    output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\">\n")
                curr_image_id += 1
                curr_entry = 1
            elif "<entry>" in line:
                in_entry = True
                output.write("<entry id=\"" + str(curr_entry) + "\">\n")
                curr_entry += 1            
            elif "</entry>" in line:
                in_entry = False
                output.write("</entry>\n")
            elif in_entry:
                output.write(line)
            elif "</item>" in line:
                output.write("</image>\n")            
        
        output.write("</volume>\n")
        output.write("</ssda>")
        
        inp.close()
        output.close()   
    
    elif transcription_type == "markup v1":
        inp = open(path_to_transcription,'r',encoding="utf-8")        
        typ = input("Enter record type: ")
        country = input("Enter country: ")
        state = input("Enter first-level administrative division: ")
        city = input("Enter city: ")
        institution = input("Enter institution: ")
        output_dir = input("Enter output directory: ")        
        volume_identifier = input("Enter the volume's unique identifier: ")        
        volume_title = input("Enter the volume's title: ")
        img_type = input("Enter image file extensions: ")
        curr_image_id = int(input("Enter first transcribed image file name: "))        
        
        output = open(output_dir,'w',encoding="utf-8")
        output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")    
        output.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
        output.write("<volume type=\"" + typ + "\" country=\"" + country + "\" state=\"" + state + "\" city=\"" + city + "\" institution=\"" + institution + "\" id=\"" + volume_identifier + "\" title=\"" + volume_title + "\">\n")
        
        in_entry = False
        for line in inp:
            if ("<folio>" in line) and ("</folio>" not in line):
                output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\">\n")
                curr_image_id += 1
                curr_entry = 1
            elif ("<folio>" in line) and ("</folio>" in line):
                output.write("</image>\n")
                output.write("<image id=\"" + str(curr_image_id) + "\" type=\"" + img_type + "\">\n")
                curr_image_id += 1
                curr_entry = 1
            elif "<entry>" in line:
                in_entry = True
                output.write("<entry id=\"" + str(curr_entry) + "\">\n")
                curr_entry += 1            
            elif "</entry>" in line:
                in_entry = False
                output.write("</entry>\n")
            elif in_entry:
                output.write(line)
            elif "</folio>" in line:
                output.write("</image>\n")
                
        output.write("</volume>\n")
        output.write("</ssda>")
        
        inp.close()
        output.close() 
    
    else:
        print("that transcription type is not supported yet")
        
    return

In [ ]:
#no_test

universal_markup_converter("transcriptions/seven.xml", "spatial historian")

Enter record type: baptism
Enter country: Cuba
Enter first-level administrative division: La Habana
Enter city: Havana
Enter institution: Iglesia de Jesús María y José
Enter output directory: transcriptions/8186.xml
Enter image file extensions: jpg
Enter first transcribed image file name: 1025
Does the transcription contain meaningful folio numbers? (Y/N): poop


In [ ]:
#no_test

universal_markup_converter("transcriptions/cf_1675_1717_basic.xml", "markup v1")

Enter record type: baptism
Enter country: Brazil
Enter first-level administrative division: Rio de Janeiro
Enter city: Cabo Frio
Enter institution: Igreja de Nossa Senhora da Assunção
Enter output directory: transcriptions/701075.xml
Enter the volume's unique identifier: 701075
Enter the volume's title: Book 1 of Free Baptisms, 1675 - 1717
Enter image file extensions: JPG
Enter first transcribed image file name: 1001


In [ ]:
#export

def process_brazilian_transcriptions(path_to_transcription):
    inp = open(path_to_transcription, 'r', encoding="utf-8")
    
    output = open("transcriptions/cf_1675_1717_basic.xml", 'w', encoding="utf-8")
    output.write("<?xml version=\"1.0\" encoding=\"utf-8\"?>\n")    
    output.write("<ssda xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xmlns:xsd=\"http://www.w3.org/2001/XMLSchema\">\n")
    output.write("<volume>\n")    
    
    first_entry = True
    in_entry = False
    
    for line in inp:        
        low_line = line.lower()
        words = low_line.split(' ')        
        if (("asentos" in words) or ("asentos\n" in words) or ("assentos" in words)) and ("fl" in words) and first_entry:
            first_entry = False
            output.write("<folio>\n")
        elif (("asentos" in words) or ("asentos\n" in words) or ("assentos" in words)) and ("fl" in words):            
            output.write("</folio>\n")
            output.write("<folio>\n")
        elif (in_entry == False) and ('/' in line):
            output.write("<entry>\n")
            in_entry = True
            output.write(line[1:])
        elif in_entry and ('/' in line):
            output.write(line[:line.find('/')])
            in_entry = False
            output.write("\n</entry>\n")
        elif in_entry:
            output.write(line)   
            
    output.write("</volume>\n")
    output.write("</ssda>")
    
    inp.close()
    output.close()
    
    return

In [ ]:
#no_test

process_brazilian_transcriptions("transcriptions/cf_1675_1717.txt")

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 71-relationship-builder.ipynb.
